# Fourier Decomposition
### From Waves to Images
---
**Notebook 1 — 2026**
<div style="font-size:15px; padding:8px; margin:2px; font-weight:600; background-color:#E80808; color:white;text-align:center;">
    <div style=" ">
        Introduction to Multidimensional Fourier Transform
    </div>
</div>
<div style="border-bottom: 1px gray dotted; padding:8px;margin:2px;text-align:center; font-size:15px; color:#444;">
<i>Daniel Sage — École Polytechnique Fédérale de Lausanne (EPFL)</i>
</div>

**Important notice**: These interactive notebooks complement the lecture and are not self-contained; in-class explanations are required.

In [30]:
import numpy as np, matplotlib.pyplot as plt
from skimage import data, transform, draw, io
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
from matplotlib.patches import Circle

size = 128
def fft2c(x): return np.fft.fftshift(np.fft.fft2(x))
def ifft2c(F): return np.real(np.fft.ifft2(np.fft.ifftshift(F)))
def snr(x,y): return 10*np.log10(np.mean(x*x)/(np.mean((x-y)**2)+1e-12))
def radial_grid(n):a=(np.arange(n)-n//2)/n; X,Y=np.meshgrid(a,a); return np.sqrt(X*X+Y*Y)
def showim(ax,img,title="",vmin=None,vmax=None):
    ax.imshow(img,cmap="gray",vmin=vmin,vmax=vmax); ax.set_title(title); ax.axis("off")
def showFM(ax,F,title=""):
    ax.imshow(np.log(np.abs(F)+1e-12),cmap="gray"); ax.set_title(title); ax.axis("off")

## 1.1 Wave decomposition 1D

In [31]:
N=size; t=np.linspace(-.5,.5,N,endpoint=False)
x=np.maximum(1-np.abs(t)/.1,0); x[N//4-2:N//4+2]=0.8; X=np.fft.fft(x-x.mean())
k=np.argsort(np.abs(X))[::-1]; n=np.arange(N)

@interact(K=IntSlider(min=1, max=12, step=1, value=1))
def tri(K):
    r=np.zeros(N); used=set(); 
    fig,ax=plt.subplots(2,1,figsize=(10,6))
    ax[0].plot(x,'k',lw=2)
    for i in k:
        if i==0 or i in used or (-i)%N in used: continue
        used|={i,(-i)%N}; r+=2*np.real(X[i]/N*np.exp(1j*2*np.pi*i*n/N))
        ax[0].plot(r+x.mean(),'--')
        if len(used)//2>=K: break
    ax[0].set_title(f"Reconstruction SNR={snr(x,r+x.mean()):.2f} dB")
    ax[0].grid(True, linewidth=0.5, linestyle=':'); ax[0].margins(x=0.01, y=0.01)
    for j,i in enumerate(list(used)[:K]): 
        ax[1].plot(2*np.real(X[i]/N*np.exp(1j*2*np.pi*i*n/N)))
    ax[1].grid(True, linewidth=0.5, linestyle=':'); ax[1].margins(x=0.01, y=0.01)
    plt.show()


interactive(children=(IntSlider(value=1, description='K', max=12, min=1), Output()), _dom_classes=('widget-int…

## 1.2 Single 2D wave

In [32]:
def wave2d(p,a,A=1,ph=0):
    y,x=np.meshgrid(np.arange(size),np.arange(size))
    fx,fy=np.cos(a)/p,np.sin(a)/p
    fx,fy=int(round(fx*size))/size,int(round(fy*size))/size
    return np.real(A*np.exp(1j*(2*np.pi*(fx*x+fy*y)+ph)))

@interact(period=FloatSlider(min=2, max=64, step=1, value=16),angle=FloatSlider(min=0, max=2*np.pi, step=.05, value=0))
def w(period,angle):
    W=wave2d(period,angle)
    fig,ax=plt.subplots(1,2,figsize=(8,4))
    showim(ax[0],W); showFM(ax[1],fft2c(W))
    plt.show()

interactive(children=(FloatSlider(value=16.0, description='period', max=64.0, min=2.0, step=1.0), FloatSlider(…

## 1.3 Sum of multiple waves

In [33]:
#img = data.camera()
img = io.imread('car_pad.tif')
size=128
img = transform.resize(img, (size,size), anti_aliasing=True)

F = fft2c(img); m = np.abs(F)
c = size//2; m[c,c] = 0
idx = np.argsort(m.ravel())[::-1]
pts = np.column_stack(np.unravel_index(idx, m.shape))

@interact(N=IntSlider(min=4, max=100, step=4, value=12))
def top(N):
    M = min(3*N, len(pts)); cand = pts[:M]
    rng = np.random.default_rng(); rng.shuffle(cand)

    pairs, used = [], set()
    for ky, kx in cand:
        ky2, kx2 = 2*c-ky, 2*c-kx
        if (ky,kx) in used or (ky2,kx2) in used: continue
        used |= {(ky,kx),(ky2,kx2)}
        pairs.append((ky,kx))
        if 2*len(pairs) >= N: break

    K = len(pairs)
    fig, ax = plt.subplots(K+1, 3, figsize=(8, 2*(K+1)))
    Y, X = np.meshgrid(np.arange(size), np.arange(size), indexing="ij")
    r = np.zeros_like(img)

    for i,(ky,kx) in enumerate(pairs):
        fy, fx = (ky-c)/size, (kx-c)/size
        a = 2*np.abs(F[ky,kx])/(size*size)
        w = 2*np.real(F[ky,kx]/(size*size) * np.exp(1j*2*np.pi*(fx*X + fy*Y)))
        r += w
        showim(ax[i,0], w/np.max(np.abs(w)+1e-12),f"freq={ky-size//2},{kx-size//2}",  -1, 1)
        showim(ax[i,1], w, f"|c|={a:.3e}", -1, 1)
        showim(ax[i,2], r, f"SNR={snr(img,r):.2f} dB")

    sel = np.zeros_like(m, bool)
    for ky,kx in pairs: sel[ky,kx] = sel[2*c-ky,2*c-kx] = True
    showim(ax[K,0], img, 'original')
    showFM(ax[K,1], F); ax[K,1].plot(np.where(sel)[1], np.where(sel)[0], 'r.', ms=3)
    showim(ax[K,2], r, f"SNR={snr(img,r):.2f} dB")
    plt.tight_layout(); plt.show()

interactive(children=(IntSlider(value=12, description='N', min=4, step=4), Output()), _dom_classes=('widget-in…